In [52]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import pandas as pd

In [70]:
#브라우저 열기
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url="https://factcheck.snu.ac.kr/?score=1"
driver.get(url)
driver.implicitly_wait(10)

#button click => 팝업창 닫기 
driver.find_element(By.XPATH,'/html/body/div[2]/div/div[1]/div/div/div[3]/button[1]').click()
driver.execute_script("document.body.style.zoom='50%'")
driver.implicitly_wait(10)

In [ ]:
#k->1 교차검증 k->0 최신팩트
def select_cross(k):
    select_element = driver.find_element(By.XPATH,'//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[1]/div/select[1]')
    select = Select(select_element)
    select.select_by_index(k)

def select_date():
    select_date_element = driver.find_element(By.XPATH, '//*[@id="__next"]/div/div[2]/div/div[2]/div[1]/div[2]/div[1]/select')
    select = Select(select_date_element)
    select.select_by_index(0)
    selected_option_text = select.first_selected_option.text
    last_date = selected_option_text.replace("최종 등록: ","").strip()
    return last_date
    


In [68]:
#edit1 : content={파이프라인,카테고리 분류,중복 출처, 발언제 제거} & 출처 URL 

#교차 검증 선택
select_cross(1)  

total_pages = int(input("crawling원하는 페이지 수 입력: "))
current_page = 1
current_page_num=1
articles_processed=0

titles = []
sources = []
contents = []
facts = []
links = []
last_reg_dates =[]

while current_page <= total_pages:
    
    for index in range(10):
        #해당 페이지 내 기사 제목
        articles = driver.find_elements(By.CSS_SELECTOR, '.jsx-3580705224 .fact-check-title')
        address_links = driver.find_elements(By.CSS_SELECTOR, '.jsx-3580705224 .fact-check-source .jsx-3580705224')
        # 현재 페이지에 기사가 10개 미만인 경우 반복문 종료
        if index >= len(articles):
            break

        article = articles[index]
        link = address_links[index]
        get_link = link.get_attribute("href")

        title = article.text
        titles.append(title)
        links.append(get_link)
        print(title)

        #js로 control
        driver.execute_script("arguments[0].click();", article)
        
        time.sleep(1)
        print(f"{index + 1}번째 기사 클릭 완료")
        source = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .fact-check-source').text
        fact = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .fact-dial-label-text').text
        content = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .mobile-display').text
        #last_reg_date = driver.find_element(By.CSS_SELECTOR, 'option.jsx-727853492').text
        #last_reg_date = last_reg_date.replace("최종 등록: ","").strip()
        last_reg_date=select_date()

        sources.append(source)
        contents.append(content)
        facts.append(fact)
        last_reg_dates.append(last_reg_date)

        articles_processed+=1

        driver.back()
        print(f"{articles_processed}번째 기사에서 뒤로가기 완료")
        driver.implicitly_wait(10)
        
        #교차검증
        select_cross(1)
        
        #driver.back() 초기화 방지
        if articles_processed<11:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            current_button_xpath = f"//button[contains(@class, 'page-index-button') and text()='{current_page_num}']"
            current_button = driver.find_element(By.XPATH, current_button_xpath)
            driver.execute_script("arguments[0].click();", current_button)
            time.sleep(3)
        else:
            current_page_num+=1
            articles_processed=0

    # 다음 페이지로 이동
    if current_page < total_pages:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        next_button_xpath = f"//button[contains(@class, 'page-index-button') and text()='{current_page + 1}']"
        next_button = driver.find_element(By.XPATH, next_button_xpath)
        driver.execute_script("arguments[0].click();", next_button)
        time.sleep(3)

    current_page += 1

driver.quit()

홍범도 장군이 독립군 몰살된 '자유시 참변'에 가담했다
1번째 기사 클릭 완료
1번째 기사에서 뒤로가기 완료
휴대용 방사능 측정기는 효과 있다
2번째 기사 클릭 완료
2번째 기사에서 뒤로가기 완료
여운형 선생·홍범도 장군만 건국훈장 두번 받았다
3번째 기사 클릭 완료
3번째 기사에서 뒤로가기 완료
집회 시설물 설치 시 도로점용하려면 지자체 별도 허가 필요하다
4번째 기사 클릭 완료
4번째 기사에서 뒤로가기 완료
제주 4·3사건이 북한 김일성 지시로 촉발됐다
5번째 기사 클릭 완료
5번째 기사에서 뒤로가기 완료
성남FC 의혹, 이미 수년간 수사 끝에 무혐의로 처분됐다
6번째 기사 클릭 완료
6번째 기사에서 뒤로가기 완료
이태원 참사 당일 서울 시내 모든 경찰 기동대가 촛불집회에 질서 투입됐다
7번째 기사 클릭 완료
7번째 기사에서 뒤로가기 완료
이태원 참사 현장에 우려할 정도로 많은 인파가 모였던 것은 아니었다
8번째 기사 클릭 완료
8번째 기사에서 뒤로가기 완료
핼러윈 행사 주최자가 없기 때문에 정부의 책임이 없다
9번째 기사 클릭 완료
9번째 기사에서 뒤로가기 완료
장병 전투화 310억-내복 95억 예산 삭감했다
10번째 기사 클릭 완료
10번째 기사에서 뒤로가기 완료


In [69]:
print(last_reg_dates)

['2023.08.31 14:44', '2023.09.19 15:16', '2023.09.02 08:35', '2023.06.21 09:18', '2023.02.21 13:31', '2023.02.08 21:34', '2022.11.28 14:37', '2022.11.10 08:58', '2022.11.08 16:25', '2022.09.26 14:57']


In [16]:
# #origin
# total_pages = int(input("crawling원하는 페이지 수 입력: "))
# current_page = 1
# current_page_num=1
# articles_processed=0

# titles = []
# sources = []
# contents = []
# facts = []

# while current_page <= total_pages:
#     for index in range(10):
#         #해당 페이지 내 기사 제목
#         articles = driver.find_elements(By.CSS_SELECTOR, '.jsx-3580705224 .fact-check-title')
        
#         # 현재 페이지에 기사가 10개 미만인 경우 반복문 종료
#         if index >= len(articles):
#             break

#         article = articles[index]
#         title = article.text
#         titles.append(title)
#         print(title)

#         #js로 control
#         driver.execute_script("arguments[0].click();", article)
#         time.sleep(0.5)
#         print(f"{index + 1}번째 기사 클릭 완료")

#         source = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .fact-check-source').text
#         content = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .fact-show-card-continer').text
#         fact = driver.find_element(By.CSS_SELECTOR, '.jsx-727853492 .fact-dial-label-text').text

#         print(source)
#         sources.append(source)
#         contents.append(content)
#         facts.append(fact)

#         articles_processed+=1

#         driver.back()
#         print(f"{articles_processed}번째 기사에서 뒤로가기 완료")
#         driver.implicitly_wait(10)
        
#         #driver.back() 초기화 방지
#         if articles_processed<11:
#             driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#             current_button_xpath = f"//button[contains(@class, 'page-index-button') and text()='{current_page_num}']"
#             current_button = driver.find_element(By.XPATH, current_button_xpath)
#             driver.execute_script("arguments[0].click();", current_button)
#             time.sleep(1)
#         else:
#             current_page_num+=1
#             articles_processed=0

#     # 다음 페이지로 이동
#     if current_page < total_pages:
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#         next_button_xpath = f"//button[contains(@class, 'page-index-button') and text()='{current_page + 1}']"
#         next_button = driver.find_element(By.XPATH, next_button_xpath)
#         driver.execute_script("arguments[0].click();", next_button)
#         time.sleep(3)

#     current_page += 1

# driver.quit()

종이컵을 규제하는 나라는 우리나라가 유일하다
1번째 기사 클릭 완료
출처: 11월 7일 환경부 공식 브리핑
1번째 기사에서 뒤로가기 완료
김포시, 서울 출퇴근 인구 80% 넘는다
2번째 기사 클릭 완료
출처: 유의동 국민의힘 정책위의장 발언, MBC, '김포 서울 편입' 가속 페달 밟는 여당‥근거는 무엇이었나?(2023.11.1.)
출처: 김병수 김포시장 발언, 국민의힘 10월 30일 [해결사! 김기현이 간다] 수도권 신도시 교통대책 마련 간담회 유튜브 라이브 영상
2번째 기사에서 뒤로가기 완료
빈대 방역 위해 뿌리는 규조토 분말, 인체에 해롭지 않다
3번째 기사 클릭 완료
출처: 인터넷 커뮤니티 게시물(루리웹, 2023년 11월 3일)
3번째 기사에서 뒤로가기 완료
액상형 대마는 합법이다
4번째 기사 클릭 완료
출처: 해당 이슈 관련 언론 보도(JTBC, "액상형 대마는 완전히 합법…홍대 이어 건대서도 '마약 광고'")
4번째 기사에서 뒤로가기 완료
응답자가 편향돼서 한국 매체 신뢰도 조사 결과 뺐다
5번째 기사 클릭 완료
출처: 10월 17일 문화체육관광위원회 국정감사
5번째 기사에서 뒤로가기 완료
국내 스타벅스 리프레셔 메뉴에는 실제 과일이 들어있지 않다
6번째 기사 클릭 완료
출처: [뉴스딱] "비싼 값에도 과일 없는 과일음료"…소송당한 스타벅스
6번째 기사에서 뒤로가기 완료
내년부터 인공눈물 가격이 최대 10배 비싸져 4만 원까지 오른다
7번째 기사 클릭 완료
출처: 서울경제 "4000원 인공눈물 내년부턴 4만원 되나…'진짜 눈물 날 판'"
7번째 기사에서 뒤로가기 완료
평생 투입된 시간당 소득, 개원 의사가 7급 공무원보다 적다
8번째 기사 클릭 완료
출처: '개원의사와 7급공무원의 평생 투입된 시간 당 소득 비교' 인포그래픽
출처: 정책현안보고 '세계적으로 일상화된 의사 단체행동' 13~19쪽(2020.1.22.)
출처: 데일리메디, 특별기고 '의사 소득이 7급 공무원보다 적은 이유'
8번째 기사에서 뒤로가기 완료
정치인이 출연한 유튜브 방송에 후

In [58]:
df=pd.DataFrame({
    'Title' : titles,
    'Source' : sources,
    'Source links' :  links,
    'Content' : contents,
    'Fact' : facts,
    'last register date' : last_reg_dates
})
df.to_excel('crawled_data.xlsx',index=False)